Data suspect

In [ ]:
import streamlit as st
import sqlite3
import pandas as pd
from datetime import date, timedelta # Butuh timedelta

# --- Konfigurasi halaman ---
st.set_page_config(page_title="Data Suspect", layout="wide")

from utils.ui import setup_header
setup_header()
# ==========================================================
# 🧭 1️⃣ KONFIGURASI SIDEBAR DENGAN LOGO
# ==========================================================
# --- CSS styling untuk sidebar ---
# st.markdown("""
#     <style>
#         [data-testid="stSidebar"] {
#             background-color: #f7f9fb;
#             padding-top: 0px;
#         }
#     </style>
# """, unsafe_allow_html=True)

# --- Konten Sidebar ---
# st.sidebar.image("Logo_BMKG.png", caption="Dashboard Monitoring Cuaca Ekstrem")
st.sidebar.markdown("""
    <div class="sidebar-footer" style="font-size: 12px; color: #666; text-align: center; margin-top: 400px;">
        © 2025 | BMKG Dashboard Prototype Aktualisasi Fadhilatul Istiqomah
    </div>
""", unsafe_allow_html=True)

#st.title("Data Suspect")

# --- Koneksi ke database SQLite ---
db_path = "data_salah.db"
table_name = "data_salah"
conn = sqlite3.connect(db_path)

# --- Ambil semua tanggal unik ---
tanggal_list = pd.read_sql_query(
    f"SELECT DISTINCT tanggal FROM {table_name} ORDER BY tanggal", conn
)["tanggal"].tolist()

# --- Widget pilih tanggal (kalender) ---
pilih_tanggal = st.date_input(
    "📅 Pilih tanggal:",
    value=date(2025, 1, 2), # Defaultnya bisa 2 Jan agar saat buka langsung tampil data 1 Jan
    min_value=date(2025, 1, 1),
    max_value=date(2025, 12, 31)
)

# --- INTI LOGIKANYA ADA DI SINI ---
# 1. Ambil tanggal yang dipilih di UI, lalu kurangi satu hari.
#    Jika Anda pilih 2 Jan, variabel ini akan berisi 1 Jan.
tanggal_untuk_query = pilih_tanggal - timedelta(days=1)


# --- Cari tanggal pertama dari list ---
tanggal_pertama = tanggal_list[0]

# --- Hapus pengecekan tanggal pertama, langsung gunakan query universal ---
query = f"""
SELECT station_wmo_id, NAME, jam, sandi_gts,
       Curah_Hujan, Curah_Hujan_Jam, tanggal
FROM {table_name}
WHERE (tanggal = ? AND jam != '00:00')
   OR (tanggal = date(?, '+1 day') AND jam = '00:00')
ORDER BY station_wmo_id, jam
"""
df = pd.read_sql_query(query, conn, params=(tanggal_untuk_query, tanggal_untuk_query))

# if tanggal_untuk_query.strftime('%Y-%m-%d') == tanggal_pertama:
#     query = f"""
#     SELECT station_wmo_id, NAME, jam, sandi_gts,
#            Curah_Hujan, Curah_Hujan_Jam, tanggal
#     FROM {table_name}
#     WHERE tanggal = ? AND jam != '00:00'
#     ORDER BY station_wmo_id, jam
#     """
#     df = pd.read_sql_query(query, conn, params=(tanggal_untuk_query,))
# else:
#     # 2. Query ini mengambil data berdasarkan tanggal yang sudah dikurangi satu.
#     #    Jika tanggal_untuk_query adalah 1 Jan, query akan:
#     #    - Mengambil jam selain '00:00' dari tanggal 1 Jan.
#     #    - Mengambil jam '00:00' dari tanggal (1 Jan + 1 hari), yaitu 2 Jan.
#     query = f"""
#     SELECT station_wmo_id, NAME, jam, sandi_gts,
#            Curah_Hujan, Curah_Hujan_Jam, tanggal
#     FROM {table_name}
#     WHERE (tanggal = ? AND jam != '00:00')
#        OR (tanggal = date(?, '+1 day') AND jam = '00:00')
#     ORDER BY station_wmo_id, jam
#     """
#     df = pd.read_sql_query(query, conn, params=(tanggal_untuk_query, tanggal_untuk_query))

conn.close()

# --- Sisa kode tidak perlu diubah ---
# --- (Kode visualisasi dan tabel di bawah sini sama persis) ---

# --- Hilangkan baris 00:00 jika itu satu-satunya jam di stasiun tsb ---
mask = ~(
    (df["jam"] == "00:00") &
    (df.groupby("station_wmo_id")["jam"].transform("count") == 1)
)
df = df[mask].copy()

# --- Pindahkan jam 00:00 ke paling bawah ---
df["sort_order"] = df["jam"].apply(lambda x: 1 if x == "00:00" else 0)
df = df.sort_values(by=["station_wmo_id", "sort_order", "jam"]).drop(columns="sort_order")

# --- CSS styling ---
st.markdown("""
    <style>
    table {
        table-layout: fixed;
        width: 100%;
    }
    th:nth-child(1), td:nth-child(1) {
        width: 80px !important;
        text-align: center;
    }
    th:nth-child(2), td:nth-child(2) {
        white-space: pre-wrap !important;
        word-wrap: break-word !important;
        text-align: left;
    }
    th:nth-child(3), td:nth-child(3) {
        width: 100px !important;
        text-align: center;
    }
    </style>
""", unsafe_allow_html=True)

# --- Tampilkan data per stasiun ---
for station_id, group in df.groupby(["station_wmo_id", "NAME"]):
    st.subheader(f"📡 {station_id[0]} - {station_id[1]}")

    # --- Tabel detail jam per jam ---
    df_table = group[["jam", "sandi_gts", "Curah_Hujan_Jam"]].reset_index(drop=True)
    df_table = df_table.fillna("-")
    df_table.columns = ["Jam", "Sandi Synop", "Curah Hujan"]
    st.write(df_table.to_html(index=False, escape=False), unsafe_allow_html=True)

    # --- Ringkasan per stasiun ---
    curah_hujan_00 = group.loc[group["jam"] == "00:00", "Curah_Hujan"].sum()
    curah_hujan_jam = group["Curah_Hujan_Jam"].sum()
    selisih = curah_hujan_00 - curah_hujan_jam

    df_summary = pd.DataFrame({
        "Curah Hujan 24 Jam": [curah_hujan_00 if curah_hujan_00 != 0 else "-"],
        "Curah Hujan Per-Jam": [curah_hujan_jam if curah_hujan_jam != 0 else "-"],
        "Selisih": [selisih if selisih != 0 else "-"]
    })

    # CSS khusus tabel ringkasan -> 3 kolom sama lebar
    st.markdown("""
        <style>
        .summary-table td, .summary-table th {
            text-align: center !important;
            width: 33% !important;
        }
        </style>
    """, unsafe_allow_html=True)

    st.write(df_summary.to_html(index=False, escape=False, classes="summary-table"), unsafe_allow_html=True)
    st.markdown("---")